In [83]:
import yaml
import os
from pprint import pprint
from SmartAITool.core import bprint
# Load API configuration from a YAML file
with open("R:/M/code/LLM/config.yaml", "r") as file:
    config = yaml.safe_load(file)

os.environ["LANGSMITH_TRACING"] = config.get("LANGSMITH_TRACING", "false")
os.environ["LANGSMITH_API_KEY"] = config["LANGSMITH_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['HUGGINGFACEHUB_API_TOKEN']
GPT_API_KEY = config['GPT_API_KEY']

In [84]:
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    api_key= GPT_API_KEY,
    base_url="https://api.gilas.io/v1/",
    model="gpt-4o-mini")



In [85]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    api_key= GPT_API_KEY,
    base_url="https://api.gilas.io/v1/",
    model="text-embedding-3-large" )

In [86]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print((vector_store))

In [87]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43130


In [88]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [89]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 66 sub-documents.


In [90]:
pprint(all_splits[0:3])

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from

In [91]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

BadRequestError: Error code: 400 - {'error': {'message': 'Maximum input token for the text-embedding-3-large model is 8191.', 'type': 'client_error', 'param': None, 'code': None}}

In [ ]:
# Pick one chunk to test

# Count its token length
from tiktoken import get_encoding
encoding = get_encoding("cl100k_base")
token_count = len(encoding.encode(test_chunk))
print(f"Token count of test chunk: {token_count}")

# Try embedding just this one chunk
embedding = embeddings.embed_documents([test_chunk])
print("Embedding succeeded:", embedding is not None)


Token count of test chunk: 5
Embedding succeeded: True
